# LDA TOPIC MODELING 

Спасибо индусам за это чудо. Феноменально пофикшено by SD.

https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/



__Required Libraries__

In [4]:
import pandas as pd
import numpy as np
import os


import string
import re
from pprint import pprint

# NLTK 
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
stop_words = stopwords.words('russian') #this depends on each language

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

__Loading data__

In [5]:
df = pd.read_json('C:/Users/Sergey/Desktop/work/FTD_work/FTD_work/a1.json')

In [6]:
df.shape[0]

2457

In [7]:
df.head(20)

,text
0,"Изречение, что за правым плечом человека стоит..."
1,"Сегодня соцсети являются отдельной жизнью, в к..."
2,С изобретением селфи весь мир просто сошёл с у...
3,"Удивительно, как в современном мире люди бываю..."
4,Не буду тянуть и отвечу сразу. Нет. Российских...
5,Пользователь Инстаграм под ником Averagerob уж...
6,"Дэмиен Херст в последнее время что-то затих, и..."
7,"Вы уж извините, но пока у меня не пройдёт (а у..."
8,<o>© Фото: Елена Иванова Депутаты Госдумы иниц...
9,"У каждой национальной кухни, будь то: корейска..."


In [8]:
data = df.text.values.tolist()

AttributeError: 'DataFrame' object has no attribute 'content'

In [ ]:
pprint(data[:1])

["From: lerxst@wam.umd.edu (where's my thing)\n"
 'Subject: WHAT car is this!?\n'
 'Nntp-Posting-Host: rac3.wam.umd.edu\n'
 'Organization: University of Maryland, College Park\n'
 'Lines: 15\n'
 '\n'
 ' I was wondering if anyone out there could enlighten me on this car I saw\n'
 'the other day. It was a 2-door sports car, looked to be from the late 60s/\n'
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition,\n'
 'the front bumper was separate from the rest of the body. This is \n'
 'all I know. If anyone can tellme a model name, engine specs, years\n'
 'of production, where this car is made, history, or whatever info you\n'
 'have on this funky looking car, please e-mail.\n'
 '\n'
 'Thanks,\n'
 '- IL\n'
 '   ---- brought to you by your neighborhood Lerxst ----\n'
 '\n'
 '\n'
 '\n'
 '\n']


__Tokenize words and Clean-up text__

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['from', 'lerxst', 'wam', 'umd', 'edu', 'where', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]

__Bigrams & Trigrams__

Bigrams are two words frequently occurring together in the document. Trigrams are 3 words frequently occurring.

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['from', 'lerxst', 'wam_umd', 'edu', 'where', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting_host', 'rac_wam_umd', 'edu', 'organization', 'university', 'of', 'maryland_college_park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


__Remove Stopwords, Make Bigrams and Lemmatize__

In [ ]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

__Let’s call the functions in order__

In [ ]:
#To run this code it is necessary to get the spacy moidel: "!python - m spacy download en", last depends on the used language 

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'es' model, keeping only tagger component (for efficiency)
# python3 -m spacy download es
nlp = spacy.load("ru_core_news_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['thing', 'subject', 'car', 'nntp_poste', 'host', 'park', 'line', 'wonder', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'door', 'really', 'small', 'addition', 'front_bumper', 'separate', 'rest', 'body', 'know', 'tellme', 'model', 'name', 'engine', 'spec', 'year', 'production', 'car', 'make', 'history', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst']]


__Create the Dictionary and Corpus needed for Topic Modeling__

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 5), (5, 1), (6, 2), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 2), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1)]]


If you want to see what word a given id corresponds to, pass the id as a key to the dictionary

In [ ]:
id2word[0]

'addition'

Or, you can see a human-readable form of the corpus itself.

In [ ]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:2]]

[[('addition', 1),
  ('body', 1),
  ('bring', 1),
  ('call', 1),
  ('car', 5),
  ('day', 1),
  ('door', 2),
  ('early', 1),
  ('engine', 1),
  ('enlighten', 1),
  ('front_bumper', 1),
  ('funky', 1),
  ('history', 1),
  ('host', 1),
  ('info', 1),
  ('know', 1),
  ('late', 1),
  ('lerxst', 1),
  ('line', 1),
  ('look', 2),
  ('mail', 1),
  ('make', 1),
  ('model', 1),
  ('name', 1),
  ('neighborhood', 1),
  ('nntp_poste', 1),
  ('park', 1),
  ('production', 1),
  ('really', 1),
  ('rest', 1),
  ('see', 1),
  ('separate', 1),
  ('small', 1),
  ('spec', 1),
  ('sport', 1),
  ('subject', 1),
  ('tellme', 1),
  ('thank', 1),
  ('thing', 1),
  ('wonder', 1),
  ('year', 1)],
 [('call', 2),
  ('day', 2),
  ('host', 1),
  ('line', 1),
  ('nntp_poste', 1),
  ('subject', 1),
  ('thank', 1),
  ('acceleration', 1),
  ('adapter', 1),
  ('add', 2),
  ('answer', 1),
  ('article', 1),
  ('attain', 1),
  ('base', 1),
  ('brave', 1),
  ('brief', 1),
  ('card', 1),
  ('clock', 5),
  ('cpu', 1),
  ('detai

__Building the Topic Model__

In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=4,
                                           alpha='auto',
                                           per_word_topics=True)

__View the topics in LDA model__

In [ ]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.115*"contain" + 0.053*"baseball" + 0.053*"cd" + 0.042*"male" + '
  '0.038*"thinking" + 0.031*"rich" + 0.016*"conservative" + 0.015*"sho" + '
  '0.015*"roman" + 0.011*"arbitrary"'),
 (1,
  '0.102*"child" + 0.071*"internet" + 0.047*"brain" + 0.046*"die" + '
  '0.039*"family" + 0.035*"firearm" + 0.033*"trial" + 0.029*"cop" + '
  '0.028*"wife" + 0.018*"agent"'),
 (2,
  '0.152*"com" + 0.111*"write" + 0.101*"line" + 0.094*"subject" + '
  '0.088*"article" + 0.072*"organization" + 0.056*"nntp_poste" + 0.051*"host" '
  '+ 0.023*"reply" + 0.020*"car"'),
 (3,
  '0.028*"people" + 0.026*"state" + 0.019*"gun" + 0.019*"right" + 0.013*"kill" '
  '+ 0.013*"government" + 0.011*"death" + 0.011*"speak" + 0.010*"force" + '
  '0.010*"country"'),
 (4,
  '0.085*"drive" + 0.080*"_" + 0.040*"card" + 0.036*"driver" + 0.027*"cpu" + '
  '0.021*"speed" + 0.021*"color" + 0.021*"memory" + 0.019*"fast" + '
  '0.019*"bit"'),
 (5,
  '0.243*"key" + 0.080*"chip" + 0.042*"secure" + 0.036*"secret" + 0.035*"bit" '


__Compute Model Perplexity and Coherence Score__

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -14.265987960433868

Coherence Score:  0.4776766959934333


__Visualize the topics-keywords__

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
19     0.330570  0.072851       1        1  29.557816
11     0.277712  0.030219       2        1  15.066216
16     0.280455 -0.194082       3        1  12.005946
8      0.282645  0.133167       4        1   9.204319
2      0.157024 -0.375054       5        1   8.395649
3      0.207859  0.175174       6        1   8.003150
17     0.118599  0.128007       7        1   3.740996
12     0.057983  0.065268       8        1   3.389059
4      0.029009 -0.081556       9        1   3.150099
6     -0.008574 -0.007319      10        1   2.114147
13    -0.136291 -0.009385      11        1   0.874506
1     -0.144216  0.006113      12        1   0.860913
9     -0.128191  0.029882      13        1   0.799603
10    -0.173629  0.000921      14        1   0.599540
5     -0.155118 -0.019646      15        1   0.597091
18    -0.176025  0.004577      16        1   0.565850
7     -0.194275  0.008502      17        1   0.351270
0     -0.202834  0.009108      18        1   0.331395
14    -0.198032  0.007916      19        1   0.317398
15    -0.224668  0.015339      20        1   0.075037, topic_info=          Term          Freq         Total Category  logprob  loglift
183        com  16670.000000  16670.000000  Default  30.0000  30.0000
18        line  17552.000000  17552.000000  Default  29.0000  29.0000
207      write  13452.000000  13452.000000  Default  28.0000  28.0000
35     subject  14002.000000  14002.000000  Default  27.0000  27.0000
45     article   9946.000000   9946.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
29579    booth      0.014671      1.167633  Topic20 -11.1083   2.8181
7734     glove      0.014671      1.167634  Topic20 -11.1083   2.8181
4963    bnr_ca      0.014671      1.167635  Topic20 -11.1083   2.8181
19848    ripem      0.014671      1.167811  Topic20 -11.1083   2.8179
4684        ei      0.014671      1.167765  Topic20 -11.1083   2.8180

[891 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
977       9  0.999623             _
925       7  0.994841         abuse
1392      4  0.998923        accept
3289     15  0.986241  access_digex
1476      5  0.995519       acquire
...     ...       ...           ...
40        8  0.030666          year
40       10  0.237661          year
819      10  0.993275     yesterday
629       1  0.999255         young
2289      5  0.995186          zone

[1314 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[20, 12, 17, 9, 3, 4, 18, 13, 5, 7, 14, 2, 10, 11, 6, 19, 8, 1, 15, 16])

__Building LDA Mallet Model__

The MALLET topic model package includes an extremely fast and highly scalable implementation of Gibbs sampling, efficient methods for document-topic hyperparameter optimization, and tools for inferring topics for new documents given trained models. 

In [ ]:
# Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
os.environ['MALLET_HOME'] = 'C:/Users/Sergey/Desktop/work/FTD_work/FTD_work/mallet-2.0.8'
mallet_path = 'C:/Users/Sergey/Desktop/work/FTD_work/FTD_work/mallet-2.0.8/bin/mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

In [ ]:
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

[(16,
  [('key', 0.03276350799088743),
   ('system', 0.016555657965671357),
   ('encryption', 0.012312401092116933),
   ('government', 0.00975601272977062),
   ('message', 0.009547327965497452),
   ('information', 0.009060396848860059),
   ('security', 0.009043006451837296),
   ('technology', 0.008834321687564126),
   ('public', 0.008782150496495835),
   ('chip', 0.00806914421856251)]),
 (19,
  [('food', 0.0079155672823219),
   ('study', 0.007789923357205679),
   ('effect', 0.00726640700255476),
   ('drug', 0.007182644385810613),
   ('doctor', 0.007015119152322318),
   ('year', 0.006721949993717804),
   ('medical', 0.006219374293252921),
   ('disease', 0.006198433639066884),
   ('patient', 0.006177492984880848),
   ('eat', 0.005947145788834443)]),
 (17,
  [('line', 0.044370271473075214),
   ('mail', 0.032732532265242545),
   ('subject', 0.025456163773920782),
   ('organization', 0.020471740097908322),
   ('host', 0.020204717400979084),
   ('sale', 0.019870939029817535),
   ('sell', 0.0

__How to find the optimal number of topics for LDA?__

My approach to finding the optimal number of topics is to build many LDA models with different values of number of topics (k) and pick the one that gives the highest coherence value.

Choosing a ‘k’ that marks the end of a rapid growth of topic coherence usually offers meaningful and interpretable topics. Picking an even higher value can sometimes provide more granular sub-topics.

If you see the same keywords being repeated in multiple topics, it’s probably a sign that the ‘k’ is too large.

The compute_coherence_values() (see below) trains multiple LDA models and provides the models and their corresponding coherence scores.

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=40, step=6)

In [ ]:
# Show graph
limit=40; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

NameError: name 'plt' is not defined

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

Num Topics = 2  has Coherence Value of 0.4155
Num Topics = 8  has Coherence Value of 0.4803
Num Topics = 14  has Coherence Value of 0.5391
Num Topics = 20  has Coherence Value of 0.5713
Num Topics = 26  has Coherence Value of 0.5758
Num Topics = 32  has Coherence Value of 0.5734
Num Topics = 38  has Coherence Value of 0.558


In [ ]:
# Select the model and print the topics
optimal_model = model_list[3]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

[(0,
  '0.029*"window" + 0.017*"file" + 0.015*"image" + 0.014*"version" + '
  '0.014*"program" + 0.013*"run" + 0.012*"display" + 0.011*"graphic" + '
  '0.010*"color" + 0.010*"software"'),
 (1,
  '0.158*"write" + 0.134*"article" + 0.061*"line" + 0.060*"subject" + '
  '0.044*"organization" + 0.029*"apr" + 0.014*"opinion" + 0.008*"man" + '
  '0.008*"number" + 0.007*"org"'),
 (2,
  '0.014*"question" + 0.014*"exist" + 0.011*"claim" + 0.010*"true" + '
  '0.010*"argument" + 0.009*"reason" + 0.009*"evidence" + 0.009*"people" + '
  '0.008*"science" + 0.008*"human"'),
 (3,
  '0.014*"people" + 0.010*"war" + 0.008*"israeli" + 0.008*"attack" + '
  '0.008*"armenian" + 0.007*"kill" + 0.007*"turkish" + 0.007*"government" + '
  '0.006*"land" + 0.006*"state"'),
 (4,
  '0.042*"post" + 0.028*"mail" + 0.023*"group" + 0.020*"send" + 0.018*"line" + '
  '0.017*"list" + 0.017*"internet" + 0.017*"question" + 0.017*"message" + '
  '0.016*"address"'),
 (5,
  '0.012*"people" + 0.010*"christian" + 0.009*"word" + 0.

__Finding the dominant topic in each sentence__

In [ ]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,14.0,0.1442,"car, bike, buy, ride, power, ground, wire, eng...",From: lerxst@wam.umd.edu (where's my thing)\nS...
1,1,15.0,0.1962,"drive, problem, card, system, driver, work, bi...",From: guykuo@carson.u.washington.edu (Guy Kuo)...
2,2,15.0,0.1556,"drive, problem, card, system, driver, work, bi...",From: twillis@ec.ecn.purdue.edu (Thomas E Will...
3,3,9.0,0.0966,"line, nntp_poste, host, subject, organization,...",From: jgreen@amber (Joe Green)\nSubject: Re: W...
4,4,13.0,0.1857,"file, line, set, program, number, read, follow...",From: jcm@head-cfa.harvard.edu (Jonathan McDow...
5,5,7.0,0.4363,"gun, law, state, people, fire, crime, case, we...",From: dfo@vttoulu.tko.vtt.fi (Foxvog Douglas)\...
6,6,4.0,0.1356,"post, mail, group, send, line, list, internet,...",From: bmdelane@quads.uchicago.edu (brian manni...
7,7,15.0,0.4946,"drive, problem, card, system, driver, work, bi...",From: bgrubb@dante.nmsu.edu (GRUBB)\nSubject: ...
8,8,0.0,0.1410,"window, file, image, version, program, run, di...",From: holmes7000@iscsvax.uni.edu\nSubject: WIn...
9,9,15.0,0.2586,"drive, problem, card, system, driver, work, bi...",From: kerr@ux1.cso.uiuc.edu (Stan Kerr)\nSubje...


In [ ]:
df_dominant_topic.shape

(11314, 5)

__Find the most representative document for each topic__

In [ ]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet.head()

,Topic_Num,Topic_Perc_Contrib,Keywords,Text
0,0.0,0.6759,"window, file, image, version, program, run, di...",From: dbl@visual.com (David B. Lewis)\nSubject...
1,1.0,0.5417,"write, article, line, subject, organization, a...",From: cramer@optilink.COM (Clayton Cramer)\nSu...
2,2.0,0.7046,"question, exist, claim, true, argument, reason...",From: kempmp@phoenix.oulu.fi (Petri Pihko)\nSu...
3,3.0,0.7479,"people, war, israeli, attack, armenian, kill, ...",From: Center for Policy Research <cpr@igc.apc....
4,4.0,0.7212,"post, mail, group, send, line, list, internet,...",From: jrm@elm.circa.ufl.edu (Jeff Mason)\nSubj...


__Topic distribution across documents__

In [ ]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics.head()

,Dominant_Topic,Topic_Keywords,Num_Documents,Perc_Documents
0.0,14.0,"car, bike, buy, ride, power, ground, wire, eng...",927.0,0.0819
1.0,15.0,"drive, problem, card, system, driver, work, bi...",430.0,0.0380
2.0,15.0,"drive, problem, card, system, driver, work, bi...",494.0,0.0437
3.0,9.0,"line, nntp_poste, host, subject, organization,...",459.0,0.0406
4.0,13.0,"file, line, set, program, number, read, follow...",409.0,0.0361
